In [67]:
## Import Libraries
import pandas as pd
# Import libraries
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.multiclass import OneVsOneClassifier


In [33]:
## Read the data
data = pd.read_csv('./data/clean_athlete.csv')
data.head()

,ID,Name,Age,Sex,Height,Weight,Year,Team,NOC,region,Games,Season,City,Sport,Event,Medal
0,1,A Dijiang,24,M,180,80,1992,China,CHN,China,1992 Summer,Summer,Barcelona,Basketball,Basketball Men's Basketball,None
1,2,A Lamusi,23,M,170,60,2012,China,CHN,China,2012 Summer,Summer,London,Judo,Judo Men's Extra-Lightweight,None
2,3,Gunnar Nielsen Aaby,24,M,175,70,1920,Denmark,DEN,Denmark,1920 Summer,Summer,Antwerpen,Football,Football Men's Football,None
3,4,Edgar Lindenau Aabye,34,M,175,70,1900,Denmark/Sweden,DEN,Denmark,1900 Summer,Summer,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold
4,5,Christine Jacoba Aaftink,21,F,185,82,1988,Netherlands,NED,Netherlands,1988 Winter,Winter,Calgary,Speed Skating,Speed Skating Women's 500 metres,None


In [37]:
## Dropping irrevlant columns
data = data[['Age', 'Sex', 'Height', 'Weight', 'region', 'City', 'Sport', 'Medal']]

In [38]:
data

,Age,Sex,Height,Weight,region,City,Sport,Medal
0,24,M,180,80,China,Barcelona,Basketball,None
1,23,M,170,60,China,London,Judo,None
2,24,M,175,70,Denmark,Antwerpen,Football,None
3,34,M,175,70,Denmark,Paris,Tug-Of-War,Gold
4,21,F,185,82,Netherlands,Calgary,Speed Skating,None
...,...,...,...,...,...,...,...,...
263381,29,M,179,89,Poland,Innsbruck,Luge,None
263382,27,M,176,59,Poland,Sochi,Ski Jumping,None
263383,27,M,176,59,Poland,Sochi,Ski Jumping,None
263384,30,M,185,96,Poland,Nagano,Bobsleigh,None


In [41]:
# simple check for nulls
data.isna().sum()

Age       0
Sex       0
Height    0
Weight    0
region    0
City      0
Sport     0
Medal     0
dtype: int64

In [42]:
data['Sex'].unique()

array(['M', 'F'], dtype=object)

In [43]:
data['Sport'].unique()

array(['Basketball', 'Judo', 'Football', 'Tug-Of-War', 'Speed Skating',
       'Cross Country Skiing', 'Athletics', 'Ice Hockey', 'Swimming',
       'Badminton', 'Sailing', 'Biathlon', 'Gymnastics',
       'Art Competitions', 'Alpine Skiing', 'Handball', 'Weightlifting',
       'Wrestling', 'Luge', 'Water Polo', 'Hockey', 'Rowing', 'Bobsleigh',
       'Fencing', 'Equestrianism', 'Shooting', 'Boxing', 'Taekwondo',
       'Cycling', 'Diving', 'Canoeing', 'Tennis', 'Modern Pentathlon',
       'Figure Skating', 'Golf', 'Softball', 'Archery', 'Volleyball',
       'Synchronized Swimming', 'Table Tennis', 'Nordic Combined',
       'Baseball', 'Rhythmic Gymnastics', 'Freestyle Skiing',
       'Rugby Sevens', 'Trampolining', 'Beach Volleyball', 'Triathlon',
       'Ski Jumping', 'Curling', 'Snowboarding', 'Rugby',
       'Short Track Speed Skating', 'Skeleton', 'Lacrosse', 'Polo',
       'Cricket', 'Racquets', 'Military Ski Patrol', 'Croquet',
       'Jeu De Paume', 'Roque', 'Alpinism', 'Motorb

In [44]:
data['Sport'].unique()

array(['None', 'Gold', 'Bronze', 'Silver'], dtype=object)

In [45]:
data['region'].unique()

array(['China', 'Denmark', 'Netherlands', 'USA', 'Finland', 'Norway',
       'Romania', 'Estonia', 'France', 'Morocco', 'Spain', 'Egypt',
       'Bulgaria', 'Italy', 'Chad', 'Azerbaijan', 'Sudan', 'Russia',
       'Argentina', 'Cuba', 'Belarus', 'Greece', 'Cameroon', 'Turkey',
       'Chile', 'Mexico', 'Nicaragua', 'Hungary', 'Nigeria', 'Algeria',
       'Kuwait', 'Bahrain', 'Pakistan', 'Iraq', 'Syria', 'Lebanon',
       'Qatar', 'Malaysia', 'Iran', 'Canada', 'Ireland', 'Australia',
       'South Africa', 'Eritrea', 'Tanzania', 'Jordan', 'Tunisia',
       'Libya', 'Belgium', 'Djibouti', 'Palestine', 'Comoros',
       'Kazakhstan', 'Brunei', 'Saudi Arabia', 'Maldives', 'Ethiopia',
       'United Arab Emirates', 'Yemen', 'Indonesia', 'Philippines',
       'None', 'Uzbekistan', 'Kyrgyzstan', 'Tajikistan', 'Japan',
       'Switzerland', 'Brazil', 'Germany', 'Monaco', 'Israel', 'Uruguay',
       'Sweden', 'Virgin Islands, US', 'Sri Lanka', 'Armenia',
       'Ivory Coast', 'Kenya', 'Benin', 

In [46]:
data['City'].unique()

array(['Barcelona', 'London', 'Antwerpen', 'Paris', 'Calgary',
       'Albertville', 'Lillehammer', 'Los Angeles', 'Salt Lake City',
       'Helsinki', 'Lake Placid', 'Sydney', 'Atlanta', 'Stockholm',
       'Sochi', 'Nagano', 'Turin', 'Beijing', 'Rio de Janeiro', 'Athina',
       'Squaw Valley', 'Innsbruck', 'Sarajevo', 'Mexico City', 'Munich',
       'Seoul', 'Berlin', 'Oslo', "Cortina d'Ampezzo", 'Melbourne',
       'Roma', 'Amsterdam', 'Montreal', 'Moskva', 'Tokyo', 'Vancouver',
       'Grenoble', 'Sapporo', 'Chamonix', 'St. Louis', 'Sankt Moritz',
       'Garmisch-Partenkirchen'], dtype=object)

In [47]:
# set aside and save unseen data set
data_unseen = data.sample(n=10000)
data = data.drop(data_unseen.index)
print(f'Data for model: {data.shape},\nData for unseen predictions: {data_unseen.shape}')
data_unseen.to_csv('./data/olympics_unseen.csv', index=False)

Data for model: (253386, 8),
Data for unseen predictions: (10000, 8)


In [77]:
# data.columns!='Medal'
X = data.loc[: , data.columns!='Medal']
y = data.loc[: , data.columns=='Medal']

In [78]:
y

,Medal
0,None
1,None
2,None
3,Gold
4,None
...,...
263381,None
263382,None
263383,None
263384,None


In [79]:
# split the data into training and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [81]:
# encoding 
# get the categorical and numeric column names
num_cols = X_train.select_dtypes(exclude=['object']).columns.tolist()
cat_cols = X_train.select_dtypes(include=['object']).columns.tolist()
print(num_cols, '\n', cat_cols)

['Age', 'Height', 'Weight'] 
 ['Sex', 'region', 'City', 'Sport']


In [82]:
 X_train.select_dtypes(exclude=['object']).columns.tolist()

['Age', 'Height', 'Weight']

In [83]:
# pipeline for numerical columns
num_pipe = make_pipeline(
    SimpleImputer(strategy='median'),
    StandardScaler()
)
num_pipe

Pipeline(steps=[('simpleimputer', SimpleImputer(strategy='median')),
                ('standardscaler', StandardScaler())])

In [84]:
# pipeline for categorical columns
cat_pipe = make_pipeline(
    SimpleImputer(strategy='constant', fill_value='N/A'),
    OneHotEncoder(handle_unknown='ignore', sparse=False)
)
cat_pipe

Pipeline(steps=[('simpleimputer',
                 SimpleImputer(fill_value='N/A', strategy='constant')),
                ('onehotencoder',
                 OneHotEncoder(handle_unknown='ignore', sparse=False))])

In [85]:
# combine both the pipelines
full_pipe = ColumnTransformer([
    ('num', num_pipe, num_cols),
    ('cat', cat_pipe, cat_cols)
])
full_pipe

ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(strategy='median')),
                                                 ('standardscaler',
                                                  StandardScaler())]),
                                 ['Age', 'Height', 'Weight']),
                                ('cat',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(fill_value='N/A',
                                                                strategy='constant')),
                                                 ('onehotencoder',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False))]),
                                 ['Sex', 'region', 'City', 'Sport'])])

In [99]:
clf = OneVsOneClassifier(ExtraTreesClassifier(bootstrap=False, ccp_alpha=0.0, class_weight=None,
                     criterion='gini', max_depth=None, max_features='sqrt',
                     max_leaf_nodes=None, max_samples=None,
                     min_impurity_decrease=0.0, min_samples_leaf=1,
                     min_samples_split=2, min_weight_fraction_leaf=0.0,
                     n_estimators=300, n_jobs=-1, oob_score=False, verbose=0, warm_start=False)) 

In [100]:
# build the model
et_olympics = make_pipeline(full_pipe, clf)
et_olympics

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('standardscaler',
                                                                   StandardScaler())]),
                                                  ['Age', 'Height', 'Weight']),
                                                 ('cat',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(fill_value='N/A',
                                                                                 strategy='constant')),
                                                                  ('onehotencoder',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse=False))]),
                                                  ['Sex', 'region', 'City',
                                                   'Sport'])])),
                ('onevsoneclassifier',
                 OneVsOneClassifier(estimator=ExtraTreesClassifier(n_estimators=300,
                                                                   n_jobs=-1)))])

In [101]:
%%time
# train the model
et_olympics.fit(X_train, y_train)

Wall time: 22min 52s


Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('standardscaler',
                                                                   StandardScaler())]),
                                                  ['Age', 'Height', 'Weight']),
                                                 ('cat',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(fill_value='N/A',
                                                                                 strategy='constant')),
                                                                  ('onehotencoder',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse=False))]),
                                                  ['Sex', 'region', 'City',
                                                   'Sport'])])),
                ('onevsoneclassifier',
                 OneVsOneClassifier(estimator=ExtraTreesClassifier(n_estimators=300,
                                                                   n_jobs=-1)))])

In [104]:
# make predictions on the test set
y_pred = et_olympics.predict(X_test)
y_pred

array(['None', 'None', 'None', ..., 'None', 'None', 'None'], dtype=object)

In [105]:
# Evaluate the performance of the classifier
accuracy = accuracy_score(y_test, y_pred)

print(f'Accuracy: {accuracy*100:,.1f}%')

Accuracy: 86.6%


In [106]:
# Export the pipeline to a file
joblib.dump(et_olympics, 'sk_olympics.pkl')

['sk_olympics.pkl']

In [107]:
# Saving as a Pickle file
import pickle
pickle.dump(et_olympics,open('olympics.pkl','wb'))
##model=pickle.load(open('olympics.pkl','rb'))

### Predciting against unseen Data

In [108]:
new_dat = pd.read_csv('./data/olympics_unseen.csv')
new_dat

,Age,Sex,Height,Weight,region,City,Sport,Medal
0,24,F,162,57,France,Sydney,Athletics,None
1,20,M,161,64,Brazil,London,Gymnastics,None
2,22,M,162,64,Finland,Mexico City,Gymnastics,None
3,21,M,175,70,Argentina,Los Angeles,Swimming,None
4,24,M,172,70,Ukraine,Turin,Cross Country Skiing,None
...,...,...,...,...,...,...,...,...
9995,23,M,186,92,Australia,Montreal,Water Polo,None
9996,18,F,165,62,Croatia,Turin,Alpine Skiing,None
9997,18,F,181,70,Australia,Sochi,Alpine Skiing,None
9998,30,M,183,79,UK,Roma,Canoeing,None


In [109]:
model = joblib.load('sk_olympics.pkl')
model

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('standardscaler',
                                                                   StandardScaler())]),
                                                  ['Age', 'Height', 'Weight']),
                                                 ('cat',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(fill_value='N/A',
                                                                                 strategy='constant')),
                                                                  ('onehotencoder',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse=False))]),
                                                  ['Sex', 'region', 'City',
                                                   'Sport'])])),
                ('onevsoneclassifier',
                 OneVsOneClassifier(estimator=ExtraTreesClassifier(n_estimators=300,
                                                                   n_jobs=-1)))])

In [110]:
pred = model.predict(new_dat)
pred

array(['None', 'None', 'None', ..., 'None', 'None', 'None'], dtype=object)

In [111]:
accuracy = accuracy_score(new_dat.Medal, pred)
print(f'Accuracy: {accuracy*100:,.1f}%')

Accuracy: 86.3%
